# Predicting the NBA Regular Season MVP using player statistics and machine learning 

Setup and Data import

In [1]:
# Import necessary libraries 
import numpy as np # Linear algebra package
import pandas as pd # Data processing and dataframe structure
import matplotlib.pyplot as plt # plotting package

In [2]:
mvp_votes = pd.read_csv('mvp_votings.csv', index_col = 0)
test_data = pd.read_csv('test_data.csv', index_col = 0)

In [3]:
mvp_votes.head()

,fga,fg3a,fta,per,ts_pct,usg_pct,bpm,season,player,win_pct,...,pts_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,fg_pct,fg3_pct,ft_pct,ws,ws_per_48
0,18.6,0.2,6.5,25.1,0.572,28.4,8.0,1980-81,Julius Erving,0.756098,...,24.6,8.0,4.4,2.1,1.8,0.521,0.222,0.787,13.8,0.231
1,18.3,0.9,4.0,19.9,0.528,24.3,5.1,1980-81,Larry Bird,0.756098,...,21.2,10.9,5.5,2.0,0.8,0.478,0.270,0.863,10.8,0.160
2,18.2,0.0,6.9,25.5,0.616,26.3,5.3,1980-81,Kareem Abdul-Jabbar,0.658537,...,26.2,10.3,3.4,0.7,2.9,0.574,0.000,0.766,14.3,0.230
3,19.3,0.0,10.1,25.1,0.585,27.6,3.7,1980-81,Moses Malone,0.487805,...,27.8,14.8,1.8,1.0,1.9,0.522,0.333,0.757,13.7,0.202
4,21.1,0.4,7.6,22.9,0.555,32.3,1.6,1980-81,George Gervin,0.634146,...,27.1,5.1,3.2,1.1,0.7,0.492,0.257,0.826,10.5,0.182


In [4]:
test_data.head()

,fga,fg3a,fta,per,ts_pct,usg_pct,bpm,player,win_pct,g,...,pts_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,fg_pct,fg3_pct,ft_pct,ws,ws_per_48
0,24.5,13.2,11.0,30.6,0.616,40.5,11.7,James Harden,0.646341,78,...,36.1,6.6,7.5,2.0,0.7,0.442,0.368,0.879,15.2,0.254
1,17.3,2.8,9.5,30.9,0.644,32.3,10.8,Giannis Antetokounmpo,0.731707,72,...,27.7,12.5,5.9,1.3,1.5,0.578,0.256,0.729,14.4,0.292
2,8.8,0.0,6.4,24.6,0.682,17.8,7.0,Rudy Gobert,0.609756,81,...,15.9,12.9,2.0,0.8,2.3,0.669,NaN,0.636,14.4,0.268
3,19.2,8.0,6.4,23.7,0.588,29.3,5.5,Damian Lillard,0.646341,80,...,25.8,4.6,6.9,1.1,0.4,0.444,0.369,0.912,12.1,0.205
4,21.0,9.8,7.0,23.3,0.583,29.5,5.5,Paul George,0.597561,77,...,28.0,8.2,4.1,2.2,0.4,0.438,0.386,0.839,11.9,0.201


Feature Selection 

In [5]:
# Selecting variables for model and target variable
y = mvp_votes['points_won'] # target variable set as number of votes awarded to player
y_2 = mvp_votes['award_share'] # target variable set as share of total votes 

features = ['fga', 'fg3a', 'fta', 'per', 'ts_pct', 'usg_pct', 'bpm', 'win_pct',
           'g', 'mp_per_g', 'pts_per_g', 'trb_per_g', 'ast_per_g', 'stl_per_g',
           'blk_per_g', 'fg_pct', 'fg3_pct', 'ft_pct', 'ws_per_48']
x = mvp_votes[features]

Modeling

In [6]:
# Building functions to make models reproducible over many interations
# Models to be built: Random Forest Regressor, Neural Net, and Stochastic Gradient Descent Regressor

# Random Forest Regressor model function
def mse_random_forest(num_estimators, x_train, y_train, x_test, y_test):
    rfr = RandomForestRegressor(n_estimators=num_estimators, 
                                random_state=1)
    rfr.fit(x_train, y_train)
    y_pred = rfr.predict(x_test)
    return mean_squared_error(y_test, y_pred)

# Neural Net model function
def mse_multi_layer_perceptron(num_hidden_layers, alpha_used, x_train, y_train, x_test, y_test):
    nn = MLPRegressor(solver='lbfgs', 
                      hidden_layer_sizes=num_hidden_layers, 
                      alpha=alpha_used,
                      random_state=1)
    nn.fit(x_train, y_train)
    y_pred = nn.predict(x_test)
    return mean_squared_error(y_test, y_pred)

# Stochastic Gradient Descent model function
def mse_sgd(iterations, alpha_used, x_train, y_train, x_test, y_test):
    sgd = SGDRegressor(max_iter=iterations,
                       alpha=alpha_used)
    sgd.fit(x_train, y_train)
    y_pred = sgd.predict(x_test)
    return mean_squared_error(y_test, y_pred)

In [7]:
# Import libraries for modeling and evaluation
from sklearn.model_selection import train_test_split # Splits data into training and test sets
from sklearn import preprocessing # Data scaling

# Scaling data to have a mean of 0 and variance of 1
x_scaled = preprocessing.scale(x)

# Split data into training and test sets for points won and award share
# Points won split
x_train_pw, x_test_pw, y_train_pw, y_test_pw = train_test_split(x_scaled,
                                                                y, 
                                                                random_state=1)
# Award share split
x_train_as, x_test_as, y_train_as, y_test_as = train_test_split(x_scaled,
                                                                y_2,
                                                                random_state=1)

Predicting Total Votes for MVP candidate

In [8]:
# Import libraries for modeling and evaluation
from sklearn.linear_model import SGDRegressor # Stochastic Gradient Descent
from sklearn.ensemble import RandomForestRegressor # Random Forest
from sklearn.neural_network import MLPRegressor # Multi-layer perceptron network
from sklearn.metrics import mean_squared_error # Metric used for model evaluation

# Testing for best combination of parameters for Random Forest model
estimators = [50, 100, 200, 500, 1000]
print("Random Forest Models Performance (Points Won)")
for num_estimators in estimators:
    print('Estimator: ', num_estimators, ' | Mean Squared Error: ', mse_random_forest(num_estimators, 
                                                                                      x_train_pw,
                                                                                      y_train_pw, 
                                                                                      x_test_pw, 
                                                                                      y_test_pw))
    
print('-------------------------------------------------------------', '\n')

# Testing for best combination of parameters for Neural Network model
layers = [50, 100, 150, 200, 250]
alpha_test = [0.0001, 0.0003, 0.001, 0.003]
print('Neural Network Models Performance (Points Won)')
for num_hidden_layers in layers:
    for alpha_used in alpha_test:
        print('Layer size: ', num_hidden_layers, ' | Alpha: ', alpha_used, ' | Mean Squared Error: ',
              mse_multi_layer_perceptron(num_hidden_layers, 
                                         alpha_used, 
                                         x_train_pw, 
                                         y_train_pw, 
                                         x_test_pw, 
                                         y_test_pw))
        
print('-------------------------------------------------------------', '\n')

# Testing for best combination of parameters for Stochastic Gradient Descent model
max_iterations = [500, 1000, 1500, 2000, 2500]
print('Stochastic Gradient Descent Models Performance (Points Won)')
for iterations in max_iterations:
    for alpha_used in alpha_test:
        print('Maximum iterations: ', iterations, ' | Alpha: ', alpha_used, ' | Mean Squared Error: ',
              mse_sgd(iterations,
                      alpha_used, 
                      x_train_pw, 
                      y_train_pw, 
                      x_test_pw,
                      y_test_pw))

Random Forest Models Performance (Points Won)
Estimator:  50  | Mean Squared Error:  29764.489731
Estimator:  100  | Mean Squared Error:  29760.241289531252
Estimator:  200  | Mean Squared Error:  30443.393080443748
Estimator:  500  | Mean Squared Error:  30984.786461582
Estimator:  1000  | Mean Squared Error:  30916.18229113987
------------------------------------------------------------- 

Neural Network Models Performance (Points Won)


C:\Users\Sebastian\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Layer size:  50  | Alpha:  0.0001  | Mean Squared Error:  63869.53103226186


C:\Users\Sebastian\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Layer size:  50  | Alpha:  0.0003  | Mean Squared Error:  64989.38179178904


...

C:\Users\Sebastian\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Layer size:  250  | Alpha:  0.0003  | Mean Squared Error:  65704.83572020747


C:\Users\Sebastian\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Layer size:  250  | Alpha:  0.001  | Mean Squared Error:  68117.77107412383
Layer size:  250  | Alpha:  0.003  | Mean Squared Error:  61989.268525916814
------------------------------------------------------------- 

Stochastic Gradient Descent Models Performance (Points Won)
Maximum iterations:  500  | Alpha:  0.0001  | Mean Squared Error:  35304.14454803249
Maximum iterations:  500  | Alpha:  0.0003  | Mean Squared Error:  35076.283900752154
Maximum iterations:  500  | Alpha:  0.001  | Mean Squared Error:  35869.980175415694
Maximum iterations:  500  | Alpha:  0.003  | Mean Squared Error:  35485.192503313825
Maximum iterations:  1000  | Alpha:  0.0001  | Mean Squared Error:  35076.31140093242
Maximum iterations:  1000  | Alpha:  0.0003  | Mean Squared Error:  35176.16990043975
Maximum iterations:  1000  | Alpha:  0.001  | Mean Squared Error:  35723.32861837968
Maximum iterations:  1000  | Alpha:  0.003  | Mean Squared Error:  35015.62639237775
Maximum iterations:  1500  | Alpha:  0.0

C:\Users\Sebastian\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [9]:
# Selecting best models for each method
# Random Forest Regressor
best_rfr_pw = RandomForestRegressor(n_estimators=100, random_state=1)
best_rfr_pw.fit(x_train_pw, y_train_pw)

# Neural Network
best_nn_pw = MLPRegressor(solver='lbfgs', hidden_layer_sizes=250, alpha=0.003, random_state=1)
best_nn_pw.fit(x_train_pw, y_train_pw)

# Stochastic Gradient Descent
best_sgd_pw = SGDRegressor(max_iter=1000, alpha=0.003)
best_sgd_pw.fit(x_train_pw, y_train_pw)

C:\Users\Sebastian\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


SGDRegressor(alpha=0.003, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='squared_loss', max_iter=1000,
             n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=None,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [10]:
# Clean the test data to fill in NA values with 0 before applying models
test_data['fg3_pct'] = test_data['fg3_pct'].fillna(value = 0)

# Scale test data 
test_data_scaled = preprocessing.scale(test_data[features])

In [11]:
# Test models against the 2018-19 season data to predict MVP
# Random Forest Regressor
best_rfr_pw_pred = best_rfr_pw.predict(test_data_scaled)

# Neural Network
best_nn_pw_pred = best_nn_pw.predict(test_data_scaled)

# Stochastic Gradient Descent
best_sgd_pw_pred = best_sgd_pw.predict(test_data_scaled)

# Add predictions to the test data as columns
test_data['Random Forest Points Predictions'] = best_rfr_pw_pred
test_data['Neural Network Points Predictions'] = best_nn_pw_pred
test_data['Stochastic Gradient Descent Points Predictions'] = best_sgd_pw_pred

# Print results
print(test_data[['player', 
                 'Random Forest Points Predictions', 
                 'Neural Network Points Predictions', 
                 'Stochastic Gradient Descent Points Predictions']].sort_values('Random Forest Points Predictions', 
                                                                                ascending = False))

                   player  Random Forest Points Predictions  \
1   Giannis Antetokounmpo                           998.000   
0            James Harden                           842.160   
3          Damian Lillard                           654.610   
12          Kawhi Leonard                           633.190   
5            Nikola Jokic                           622.370   
6            Kevin Durant                           618.760   
11          Stephen Curry                           617.570   
18            Joel Embiid                           367.960   
2             Rudy Gobert                           339.559   
4             Paul George                           338.400   
7            Clint Capela                           295.035   
38      Russell Westbrook                           220.235   
10         Andre Drummond                           189.860   
9          Nikola Vucevic                           182.396   
8      Karl-Anthony Towns                           171

Predicting Total Vote Share for MVP Candidate

In [12]:
# Import libraries for modeling and evaluation
from sklearn.linear_model import SGDRegressor # Stochastic Gradient Descent
from sklearn.ensemble import RandomForestRegressor # Random Forest
from sklearn.neural_network import MLPRegressor # Multi-layer perceptron network
from sklearn.metrics import mean_squared_error # Metric used for model evaluation

# Testing for best combination of parameters for Random Forest model
estimators = [50, 100, 200, 500, 1000]
print("Random Forest Models Performance (Award Share)")
for num_estimators in estimators:
    print('Estimator: ', num_estimators, ' | Mean Squared Error: ', mse_random_forest(num_estimators, 
                                                                                      x_train_as,
                                                                                      y_train_as, 
                                                                                      x_test_as, 
                                                                                      y_test_as))
    
print('-------------------------------------------------------------', '\n')

# Testing for best combination of parameters for Neural Network model
layers = [50, 100, 150, 200, 250]
alpha_test = [0.0001, 0.0003, 0.001, 0.003]
print('Neural Network Models Performance (Award Share)')
for num_hidden_layers in layers:
    for alpha_used in alpha_test:
        print('Layer size: ', num_hidden_layers, ' | Alpha: ', alpha_used, ' | Mean Squared Error: ',
              mse_multi_layer_perceptron(num_hidden_layers, 
                                         alpha_used, 
                                         x_train_as, 
                                         y_train_as, 
                                         x_test_as, 
                                         y_test_as))
        
print('-------------------------------------------------------------', '\n')

# Testing for best combination of parameters for Stochastic Gradient Descent model
max_iterations = [500, 1000, 1500, 2000, 2500]
print('Stochastic Gradient Descent Models Performance (Award Share)')
for iterations in max_iterations:
    for alpha_used in alpha_test:
        print('Maximum iterations: ', iterations, ' | Alpha: ', alpha_used, ' | Mean Squared Error: ',
              mse_sgd(iterations,
                      alpha_used, 
                      x_train_as, 
                      y_train_as, 
                      x_test_as,
                      y_test_as))

Random Forest Models Performance (Award Share)
Estimator:  50  | Mean Squared Error:  0.026498776674885937
Estimator:  100  | Mean Squared Error:  0.025877537932709395
Estimator:  200  | Mean Squared Error:  0.025475495629024653
Estimator:  500  | Mean Squared Error:  0.024797095665827825
Estimator:  1000  | Mean Squared Error:  0.024460382923049064
------------------------------------------------------------- 

Neural Network Models Performance (Award Share)


C:\Users\Sebastian\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Layer size:  50  | Alpha:  0.0001  | Mean Squared Error:  0.06222438530378529


C:\Users\Sebastian\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Layer size:  50  | Alpha:  0.0003  | Mean Squared Error:  0.06447636133875162


...

Layer size:  100  | Alpha:  0.001  | Mean Squared Error:  0.04506435021636346


C:\Users\Sebastian\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Layer size:  100  | Alpha:  0.003  | Mean Squared Error:  0.04470430854631221
Layer size:  150  | Alpha:  0.0001  | Mean Squared Error:  0.03870058334192173
Layer size:  150  | Alpha:  0.0003  | Mean Squared Error:  0.03982657212631944
Layer size:  150  | Alpha:  0.001  | Mean Squared Error:  0.04133281722741805


C:\Users\Sebastian\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Layer size:  150  | Alpha:  0.003  | Mean Squared Error:  0.040423315272221795
Layer size:  200  | Alpha:  0.0001  | Mean Squared Error:  0.04685743080061438
Layer size:  200  | Alpha:  0.0003  | Mean Squared Error:  0.046639791531100504
Layer size:  200  | Alpha:  0.001  | Mean Squared Error:  0.047759359624096206
Layer size:  200  | Alpha:  0.003  | Mean Squared Error:  0.048289706791704703
Layer size:  250  | Alpha:  0.0001  | Mean Squared Error:  0.042140183764964215
Layer size:  250  | Alpha:  0.0003  | Mean Squared Error:  0.039799577070629144
Layer size:  250  | Alpha:  0.001  | Mean Squared Error:  0.041665647881003784
Layer size:  250  | Alpha:  0.003  | Mean Squared Error:  0.040587140198458514
------------------------------------------------------------- 

Stochastic Gradient Descent Models Performance (Award Share)
Maximum iterations:  500  | Alpha:  0.0001  | Mean Squared Error:  0.02847753411245988
Maximum iterations:  500  | Alpha:  0.0003  | Mean Squared Error:  0.02732

In [13]:
# Selecting best models for each method
# Random Forest Regressor
best_rfr_as = RandomForestRegressor(n_estimators=1000, random_state=1)
best_rfr_as.fit(x_train_as, y_train_as)

# Neural Network
best_nn_as = MLPRegressor(solver='lbfgs', hidden_layer_sizes=150, alpha=0.0001, random_state=1)
best_nn_as.fit(x_train_as, y_train_as)

# Stochastic Gradient Descent
best_sgd_as = SGDRegressor(max_iter=1000, alpha=0.0001)
best_sgd_as.fit(x_train_as, y_train_as)

SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='squared_loss', max_iter=1000,
             n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=None,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [14]:
# Test models against the 2018-19 season data to predict MVP
# Random Forest Regressor
best_rfr_as_pred = best_rfr_as.predict(test_data_scaled)

# Neural Network
best_nn_as_pred = best_nn_as.predict(test_data_scaled)

# Stochastic Gradient Descent
best_sgd_as_pred = best_sgd_as.predict(test_data_scaled)

# Add predictions to the test data as columns
test_data['Random Forest Award Share Predictions'] = best_rfr_as_pred
test_data['Neural Network Award Share Predictions'] = best_nn_as_pred
test_data['Stochastic Gradient Descent Award Share Predictions'] = best_sgd_as_pred

# Print results
print(test_data[['player', 
                 'Random Forest Award Share Predictions', 
                 'Neural Network Award Share Predictions', 
                 'Stochastic Gradient Descent Award Share Predictions']].sort_values('Random Forest Award Share Predictions', 
                                                                                ascending = False))

                   player  Random Forest Award Share Predictions  \
1   Giannis Antetokounmpo                               0.843771   
0            James Harden                               0.774902   
12          Kawhi Leonard                               0.670114   
6            Kevin Durant                               0.617993   
11          Stephen Curry                               0.605497   
5            Nikola Jokic                               0.514549   
3          Damian Lillard                               0.508508   
2             Rudy Gobert                               0.340524   
18            Joel Embiid                               0.310957   
4             Paul George                               0.279204   
38      Russell Westbrook                               0.237782   
7            Clint Capela                               0.235065   
8      Karl-Anthony Towns                               0.163901   
9          Nikola Vucevic                       